In [4]:
import csv
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import helpers as analysis
from numpy import linalg as LA
import numpy as np

[nltk_data] Downloading package punkt to /Users/Home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df = pd.read_csv('data/coffee_fix.csv')
df

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,agtron,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,50/73,9.0,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,54/77,9.0,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,54/70,9.0,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,53/79,8.0,8.0,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,48/70,9.0,9.0,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,https://www.coffeereview.com/review/finca-patz...,\n\n\n92\n\n\nEl Gran Cafe\nFinca Patzibir\n\...,92,El Gran Cafe,Finca Patzibir,"Antigua, Guatemala","Lake Atitlán growing region, Guatemala",Medium-Light,$16.00/12 ounces,November 2022,56/74,8.0,8.0,9.0,9.0,8.0,NaN,"Crisply sweet, nut-toned. Almond brittle, pie ...",Produced by Andres Fahsen of Finca Patzibar en...,"A quietly confident, sweetly nut-toned Guatema..."
2278,https://www.coffeereview.com/review/proyecto-a...,\n\n\n93\n\n\nEl Gran Cafe\nProyecto Aurora\n...,93,El Gran Cafe,Proyecto Aurora,"Antigua, Guatemala","Antigua growing region, Guatemala",Medium-Light,$10.00/12 ounces,November 2022,58/76,9.0,9.0,8.0,9.0,8.0,NaN,"Chocolaty, floral-framed. Cocoa nib, honeysuck...",Produced at Beneficio La Esperanza from trees ...,"A deeply floral, richly chocolaty Guatemala cu..."
2279,https://www.coffeereview.com/review/finca-el-p...,\n\n\n93\n\n\nEl Gran Cafe\nFinca El Potrero\...,93,El Gran Cafe,Finca El Potrero,"Antigua, Guatemala","Antigua growing region, Guatemala",Medium-Light,$10.00/12 ounces,November 2022,60/76,9.0,9.0,8.0,9.0,8.0,NaN,"High-toned, enticingly sweet. Black cherry, na...",Produced by Josué Morales of Finca El Potrero...,"A bright, balanced, juicy Guatemala cup driven..."
2280,https://www.coffeereview.com/review/chacaya-sa...,\n\n\n93\n\n\nEl Gran Cafe\nChacayá Santiago ...,93,El Gran Cafe,Chacayá Santiago Atitlán,"Antigua, Guatemala","Lake Atitlán growing region, Guatemala",Medium-Light,$16.00/12 ounces,November 2022,55/73,9.0,8.0,8.0,9.0,8.0,NaN,"Vibrantly sweet, subtly nuanced. Apricot, dar...",Produced by smallholding farmers of Asociación...,"Balanced, brig

In [61]:
combined_descriptions = df[["desc_1"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_names = df[["name"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_locs = df[["origin"]].apply(lambda x : ' '.join(x.dropna()), axis=1)

combined_descriptions = [x for x in combined_descriptions]
combined_names = [x + " from " + combined_locs[i] for i, x in enumerate(combined_names)]

vectorizer = TfidfVectorizer()


#Replace query with the user query here 
query = ["Nutty and Sweet"]
# query = query.lower()
# query = query.split(" ")

doc_vectors  = vectorizer.fit_transform(query + combined_descriptions).toarray()

index_to_vocab = {i:v for i, v in enumerate(vectorizer.get_feature_names_out())}
doc_to_index = {v:i for i, v in enumerate(combined_names)}
index_to_doc_descriptions = {i:{'name': v, 'description': combined_descriptions[i]}for i, v in enumerate(combined_names)}

cosineSims = (np.dot(doc_vectors[0], np.transpose(doc_vectors[1:]))/(LA.norm(doc_vectors[0])* LA.norm(doc_vectors[1:])))

cosineSims = [(x, i) for i, x in enumerate(cosineSims)]

cosineSims = sorted(cosineSims, key = lambda x:x[0], reverse = True)

topTen = cosineSims[:10]
answer  = []
for sim, index in topTen: 
  answer.append((index_to_doc_descriptions[index], sim))
















[({'name': 'Guatemala CODECH Women’s Lot from Concepción Huista, Huehuetenango Department, Guatemala', 'description': 'Rich-toned, sweetly nutty. Almond butter, baking chocolate, date, magnolia, cedar in aroma and cup. Sweet structure with gently brisk acidity; full, satiny'}, 0.009803247935896589), ({'name': 'Guatemala Washed Finca El General Pache from Acatenango growing region, Guatemala', 'description': 'Sweet-toned, richly nutty. Cashew butter, date, baker’s chocolate, magnolia, oak in aroma and cup. Sweet structure with brisk acidity; satiny-smooth mouthfeel. The richly drying finish consolidates around notes of cashew and magnolia.'}, 0.007572892243288811), ({'name': 'Philippines Sitio Belis 1017 from Benguet, Philippines', 'description': 'Sweetly pungent with savory undertones. Lychee, marjoram, cocoa nib, freesia-like flowers, roasted almond in aroma and cup. Crisply sweet structure with juicy acidity; smooth, satiny mouthfeel. Nutty, sweetly herbaceous finish.'}, 0.0075516509

In [46]:
search_terms = "my query"
documents = ["my","list","of","docs"]

vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform([search_terms] + documents).toarray()

print(np.dot(doc_vectors[0], np.transpose(doc_vectors[1:]))/(LA.norm(doc_vectors[0])* LA.norm(doc_vectors[1:])))

# document_scores = [item.item() for item in cosine_similarities[1:]]

[0.31395688 0.         0.         0.        ]
